In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#!pip install pycaret
from pycaret.classification import *
#from pycaret.regression import *

# Importing the data - only use *.MLReady.csv files!!!

In [5]:
df=pd.read_csv("../../../../Downloads/FM_Acq_Perf_FMAC_EPOCH2_ML.csv")
df.head()

,Unnamed: 0,Unnamed: 0.1,id,origChannel,origIntRate,origUPB,origDate,firstPmtDate,origLTV,numBorrowers,...,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct
0,1133339,1133339,100209956357,2,5.125,348000,2009-02-01,2009-04-01,87.0,1.0,...,5.04,5.13,0.041667,2.87,-0.125,-0.085,-0.005,-0.023810,-0.016865,-0.000975
1,1133340,1133340,100392775700,3,4.625,195000,2009-02-01,2009-04-01,52.0,2.0,...,5.04,5.13,0.041667,2.87,-0.625,0.415,-0.505,-0.119048,0.082341,-0.098441
2,1133341,1133341,100621428854,2,4.875,342000,2009-02-01,2009-04-01,80.0,1.0,...,5.04,5.13,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708
3,1133342,1133342,100720959726,1,5.375,93000,2009-02-01,2009-05-01,70.0,1.0,...,5.04,5.13,0.041667,2.87,0.125,-0.335,0.245,0.023810,-0.066468,0.047758
4,1133343,1133343,100783704799,1,4.875,182000,2009-02-01,2009-04-01,76.0,2.0,...,5.04,5.13,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 60 columns):
Unnamed: 0                    119696 non-null int64
Unnamed: 0.1                  119696 non-null int64
id                            119696 non-null int64
origChannel                   119696 non-null int64
origIntRate                   119696 non-null float64
origUPB                       119696 non-null int64
origDate                      119696 non-null object
firstPmtDate                  119696 non-null object
origLTV                       119696 non-null float64
numBorrowers                  119696 non-null float64
origDebtIncRatio              119696 non-null float64
borrCreditScore               119696 non-null float64
loanPurp                      119696 non-null int64
zipCode                       119696 non-null int64
pMIperct                      119696 non-null float64
mortInsType                   119696 non-null float64
bestCreditScore               119696 non-

## Remove columns not involved in 'Foreclosure or Not' query

In [7]:
df.drop(['Unnamed: 0','Unnamed: 0.1','id','firstPmtDate','rptPeriod','currIntRate','currUPB','loanAge','monMatur',\
         'zeroBalDate','lastPdInstDate','forecloDate','dispDate','forecloCost','propRepCost','recovCosts',\
         'miscCost','holdTaxCost','saleProceed','credEnhProceed','repurchProceed','otherForecloProceed',\
         'nonIntUPB','prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator',\
         'deliqGood','deliqBad','deliqMax'],1,inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 30 columns):
origChannel           119696 non-null int64
origIntRate           119696 non-null float64
origUPB               119696 non-null int64
origDate              119696 non-null object
origLTV               119696 non-null float64
numBorrowers          119696 non-null float64
origDebtIncRatio      119696 non-null float64
borrCreditScore       119696 non-null float64
loanPurp              119696 non-null int64
zipCode               119696 non-null int64
pMIperct              119696 non-null float64
mortInsType           119696 non-null float64
bestCreditScore       119696 non-null float64
worstCreditScore      119696 non-null float64
avgCreditScore        119696 non-null float64
bankNumber            119696 non-null int64
stateNumber           119696 non-null int64
mSA                   119696 non-null int64
zeroBalCode           119696 non-null float64
fmacRateMax           119696 n

## Engineer originDate into Date features

In [10]:
df['origDate'] = pd.to_datetime(df['origDate'])

In [11]:
df['origYear'] = df['origDate'].dt.year
df['origMonth'] = df['origDate'].dt.month

In [12]:
df.drop(['origDate'],1,inplace=True)

In [13]:
df[['origYear','origMonth']].head()

,origYear,origMonth
0,2009,2
1,2009,2
2,2009,2
3,2009,2
4,2009,2


# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [14]:
df.zeroBalCode.unique()

array([ 9.,  1.,  3., 16.,  6.,  2., 15.])

In [15]:
df['zeroBalCode'] = df.zeroBalCode.replace(1,0)
df['zeroBalCode'] = df.zeroBalCode.replace([9,3,16,6,2,15],1)

In [16]:
df.zeroBalCode.unique()

array([1., 0.])

In [17]:
#Checking datatypes of individual feature
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
origChannel           119696 non-null int64
origIntRate           119696 non-null float64
origUPB               119696 non-null int64
origLTV               119696 non-null float64
numBorrowers          119696 non-null float64
origDebtIncRatio      119696 non-null float64
borrCreditScore       119696 non-null float64
loanPurp              119696 non-null int64
zipCode               119696 non-null int64
pMIperct              119696 non-null float64
mortInsType           119696 non-null float64
bestCreditScore       119696 non-null float64
worstCreditScore      119696 non-null float64
avgCreditScore        119696 non-null float64
bankNumber            119696 non-null int64
stateNumber           119696 non-null int64
mSA                   119696 non-null int64
zeroBalCode           119696 non-null float64
fmacRateMax           119696 non-null float64
fmacRateMin           119696 

# Review values for dates and slice (if needed) the timeframe you want

In [18]:
df.origYear.unique()

array([2009, 2010, 2011, 2012, 2013], dtype=int64)

In [19]:
df['origMonth'].unique()

array([ 2,  3,  5,  4,  6,  7,  8,  9, 10, 11, 12,  1], dtype=int64)

# Coerce the column dtype (int in this case)

In [20]:
df = df.astype({'origLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'borrCreditScore':'int','mortInsType':'int','bestCreditScore':'int',\
               'worstCreditScore':'int','avgCreditScore':'int','zeroBalCode':'object'})
#,'zeroBalCode':'int'

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
origChannel           119696 non-null int64
origIntRate           119696 non-null float64
origUPB               119696 non-null int64
origLTV               119696 non-null int32
numBorrowers          119696 non-null int32
origDebtIncRatio      119696 non-null int32
borrCreditScore       119696 non-null int32
loanPurp              119696 non-null int64
zipCode               119696 non-null int64
pMIperct              119696 non-null float64
mortInsType           119696 non-null int32
bestCreditScore       119696 non-null int32
worstCreditScore      119696 non-null int32
avgCreditScore        119696 non-null int32
bankNumber            119696 non-null int64
stateNumber           119696 non-null int64
mSA                   119696 non-null int64
zeroBalCode           119696 non-null object
fmacRateMax           119696 non-null float64
fmacRateMin           119696 non-null float64


In [22]:
#df.to_csv('D:\FM_FULL_EPOCH2_MLReady.csv')

# Review metrics last time before run (to have record of starting values)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119696 entries, 0 to 119695
Data columns (total 31 columns):
origChannel           119696 non-null int64
origIntRate           119696 non-null float64
origUPB               119696 non-null int64
origLTV               119696 non-null int32
numBorrowers          119696 non-null int32
origDebtIncRatio      119696 non-null int32
borrCreditScore       119696 non-null int32
loanPurp              119696 non-null int64
zipCode               119696 non-null int64
pMIperct              119696 non-null float64
mortInsType           119696 non-null int32
bestCreditScore       119696 non-null int32
worstCreditScore      119696 non-null int32
avgCreditScore        119696 non-null int32
bankNumber            119696 non-null int64
stateNumber           119696 non-null int64
mSA                   119696 non-null int64
zeroBalCode           119696 non-null object
fmacRateMax           119696 non-null float64
fmacRateMin           119696 non-null float64


In [24]:
df.head()

,origChannel,origIntRate,origUPB,origLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,zipCode,pMIperct,...,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
0,2,5.125,348000,87,1,50,689,2,51,25.0,...,0.041667,2.87,-0.125,-0.085,-0.005,-0.023810,-0.016865,-0.000975,2009,2
1,3,4.625,195000,52,2,54,703,1,82,0.0,...,0.041667,2.87,-0.625,0.415,-0.505,-0.119048,0.082341,-0.098441,2009,2
2,2,4.875,342000,80,1,54,746,1,981,0.0,...,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2
3,1,5.375,93000,70,1,50,780,1,496,0.0,...,0.041667,2.87,0.125,-0.335,0.245,0.023810,-0.066468,0.047758,2009,2
4,1,4.875,182000,76,2,22,802,1,18,0.0,...,0.041667,2.87,-0.375,0.165,-0.255,-0.071429,0.032738,-0.049708,2009,2


# Pycaret - Setup with categorical definition

### Use feature engineering, polynomial features, trig feats, PCA

In [25]:
# models=setup(df, target = 'rateDiffPct',
#              feature_interaction=True, feature_ratio=True,
#              polynomial_features=True, pca=True, pca_components=10,
#             numeric_features = ['origUPB','origLTV','numBorrowers','origDebtIncRatio',\
#                                'avgCreditScore'],
#             ignore_features = ['origIntRate','rank','deal'])

### Use feature engineering, polynomial features, trig feats

In [26]:
models=setup(df, target = 'zeroBalCode')

 
Setup Succesfully Completed!


,Description,Value
0,session_id,6412
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(119696, 31)"
4,Missing Values,False
5,Numeric Features,26
6,Categorical Features,3
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


## Compare Models with blacklist exclusions

In [27]:
model_results=compare_models() #blacklist = ['tr','ransac'],'lar','par','huber','llar','lasso','en','ridge','omp','br','svm'])
model_results

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,CatBoost Classifier,0.8647,0.7895,0.0915,0.5196,0.1555,0.1196
1,Gradient Boosting Classifier,0.8645,0.7882,0.0521,0.5328,0.0947,0.0721
2,Ridge Classifier,0.8638,0,0,0,0,0
3,Extreme Gradient Boosting,0.8638,0.7865,0.0289,0.5048,0.0546,0.0405
4,Logistic Regression,0.8632,0.7308,0.005,0.3296,0.0099,0.0061
5,Linear Discriminant Analysis,0.8632,0.7557,0.0184,0.4544,0.0353,0.0249
6,Light Gradient Boosting Machine,0.8632,0.7873,0.0655,0.4851,0.1152,0.0857
7,Extra Trees Classifier,0.8622,0.7353,0.023,0.3935,0.0434,0.029
8,Ada Boost Classifier,0.8618,0.7747,0.0696,0.4531,0.1206,0.0876
9,SVM - Linear Kernel,0.859,0,0.0359,0.2894,0.0549,0.0351


## Create models - store in variable

In [28]:
catboost =create_model('catboost', fold = 10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,0.8655,0.7703,0.0897,0.5395,0.1538,0.1196
1,0.8661,0.8260,0.0985,0.5488,0.1670,0.1309
2,0.8678,0.7918,0.0963,0.5946,0.1657,0.1328
3,0.8643,0.7891,0.0985,0.5114,0.1651,0.1267
4,0.8615,0.7686,0.0768,0.4487,0.1311,0.0951
5,0.8669,0.8042,0.0987,0.5625,0.1679,0.1327
6,0.8657,0.7868,0.0833,0.5429,0.1445,0.1123
7,0.8633,0.7955,0.1053,0.4898,0.1733,0.1315
8,0.8612,0.7756,0.0722,0.4459,0.1243,0.0897
9,0.8642,0.7866,0.0963,0.5116,0.1621,0.1242


In [ ]:
catboost =create_model('catboost', fold = 50)

### Catboost fold results (not shown - reran in same cell)
* 10   0.5?
* 20   0.5633
* 50   0.5633

In [ ]:
lightgbm = create_model('lightgbm', fold=50)

### LightGBM fold results (not shown - reran in same cell)
    * 10   0.5?
    * 20   0.5?
    * 50   0.5585

In [ ]:
xgboost = create_model('xgboost', fold = 50)

## Tune Models

In [ ]:
tuned_catboost = tune_model('catboost') # from fold=50

* Made no difference creating the model with fold= 10 vs. fold = 50 so use fold = 10

In [ ]:
tuned_lightgbm = tune_model('lightgbm')

In [ ]:
tuned_xgboost = tune_model('xgboost')

## Ensemble models

In [ ]:
catboost_bagged = ensemble_model(catboost)

In [ ]:
catboost_boosted = ensemble_model(catboost)

## Blend 3 chosen Models

#### First blend 'non-tuned' models

In [ ]:
blend3 = blend_models(estimator_list = [catboost, lightgbm, xgboost])

#### Next blend tuned models

In [ ]:
blendTuned3 = blend_models(estimator_list = [tuned_catboost, tuned_lightgbm, tuned_xgboost])

## Evaluate Models

In [ ]:
plot_model(lightgbm, plot = 'residuals')

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:
plot_model(lightgbm, plot='vc')

In [ ]:
plot_model(lightgbm, plot = 'feature')

In [ ]:
interpret_model(lightgbm, plot='correlation', feature='avgCreditScore')

In [ ]:
interpret_model(tuned_catboost)

In [ ]:
save_model()

In [ ]:
interpret_model(estimator=logreg_model, plot='reason')

In [ ]:
logreg_model2=create_model('xgboost')

In [ ]:
plot_model(estimator=logreg_model2, plot='feature')

In [ ]:
plot_model(estimator=logreg_model2)

In [ ]:
tunned_logreg_model=tune_model('catboost')

In [ ]:
interpret_model(estimator=tunned_logreg_model,plot='summary')

In [ ]:
plot_model(estimator=tunned_logreg_model,plot='feature')

In [ ]:
interpret_model(estimator=catboost, plot='correlation')

In [ ]:
interpret_model(estimator=catboost,plot='summary')